<a href="https://colab.research.google.com/github/Ryoya1012/Data-Analysis/blob/main/10ms_60ms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np

# エクセルファイルの読み込み
file_path = 'Left_test1.xlsx'  # ここをエクセルファイルのパスに置き換えてください
data = pd.read_excel(file_path)

# データの取得
time1 = data['Time1'].values  # 10msのサンプリングレートのデータ
data1 = data['Data1'].values
time2 = data['Time2'].values  # 60msのサンプリングレートのデータ
data2 = data['Data2'].values

# 重複と欠損値のチェックと除去
def clean_data(time, data):
    df = pd.DataFrame({'Time': time, 'Data': data})
    df = df.drop_duplicates(subset='Time')
    df = df.dropna()
    df = df[df['Time'] != np.inf]
    df = df[df['Data'] != np.inf]
    return df['Time'].values, df['Data'].values

time1, data1 = clean_data(time1, data1)
time2, data2 = clean_data(time2, data2)

# 時間の共通軸を作成 (10msのサンプリングレートで共通軸を作成)
start_time = max(min(time1), min(time2))
end_time = min(max(time1), max(time2))
sampling_rate = 0.01  # 10ms (0.01秒)
common_time = np.arange(start_time, end_time, sampling_rate)

# アップサンプリングのための補間関数を作成
interp_data1 = interp1d(time1, data1, kind='linear', fill_value="extrapolate")
interp_data2 = interp1d(time2, data2, kind='linear', fill_value="extrapolate")

# 共通時間軸に合わせてデータを補間
upsampled_data1 = interp_data1(common_time)
upsampled_data2 = interp_data2(common_time)

# 結果をデータフレームにまとめる
result_df = pd.DataFrame({
    'Common Time': common_time,
    'Upsampled Data1': upsampled_data1,
    'Upsampled Data2': upsampled_data2
})

# 結果をエクセルファイルに保存
result_df.to_excel('Left2_fix.xlsx', index=False)
